In [17]:
''' Gestion de l input, lecture du fichier excel '''

import pandas as pd

DATASET_FILE = "restaurants-without-IDs.xlsx"
SORTED_FILE = "restaurants-sorted.csv"
SORTING_COLUMN = "name"
SHEET_TO_ANALYZE = "Sheet1"
WINDOW_SIZE = 3
DUPLICATE_FILE = "restaurants-duplicates.xlsx"

def sortFile(inputFile,sortingCol,sheetName,outputFile):
    xl = pd.ExcelFile(inputFile)
    df = xl.parse(sheetName)
    df[sortingCol] = df[sortingCol].astype(str)
    df = df.sort_values(sortingCol)
    df.to_csv(outputFile,index=False)
    
sortFile(DATASET_FILE,SORTING_COLUMN,SHEET_TO_ANALYZE,SORTED_FILE)
''' On observe que les données sont dans l ordre mais rentrees plusieurs fois '''
'''Sort par adress ou name sont interessants'''

'Sort par adress ou name sont interessants'

In [18]:
''' Definition des mesures de similarite/distance'''
import nltk

pd.set_option('display.max_rows', None)

'''En utilisant Jaccard et les champs name et adress
Les champs city et type ne semblent pas tres pertinents car compose d un ou deux mots
City pourraient etre pertinent si les deux city sont un peu similaires et que les deux premiers champ sont aussi similaires'''


def comparePair1(rec1,rec2):
    n1 = set(rec1['name'])
    n2 = set(rec2['name'])
    a1 = set(rec1['address'])
    a2 = set(rec2['address']) 
    dname = nltk.jaccard_distance(n1,n2)
    dadress = nltk.jaccard_distance(a1,a2)
    if dname < 0.2 and dadress < 0.2:
        return True
    return False



'''Gestion de la fenetre, calcul de la similarite selon la fonction definissant si deux records sont similaires
puis transitive closure step'''
def sortedNeighboorProgram(fileSorted,windowSize,sheetName,similarityFunction,outputFile):
    df = pd.read_csv(fileSorted)
    
    df['Similar to'] = ''
    df.info()
    numberOfRows = len(df)+1
    # Creation de la fenetre
    for i in range(numberOfRows-windowSize):
        window = df.iloc[i:i+windowSize]
        window.index.name = 'index'
        #Pour la premiere fenetre, faire toutes les comparaisons deux à deux
        if i == 0:
            for j in range(windowSize):
                for k in range(j+1,windowSize):
                    areSimilar = similarityFunction(window.iloc[j],window.iloc[k])
                    #S'ils sont similaires, ajout de chacun à la liste des similaires de l'autre
                    if areSimilar:
                        window.at[j,'Similar to']  = window.at[j,'Similar to'] + ","  + str(k)
                        window.at[k,'Similar to']  = window.at[k,'Similar to'] + "," + str(j)
        #Sinon faire les comparaisons du nouveau record avec tous les anciens records de la fenetre
        else:
            lastRow = window.iloc[windowSize-1]
            lastRowIndex = window.index[windowSize-1]
            for j in range(i,i+windowSize-1):
                areSimilar = similarityFunction(lastRow,window.loc[j])
                if areSimilar:
                    window.at[j,'Similar to']  = window.at[j,'Similar to'] + "," + str(lastRowIndex)
                    lastRow['Similar to']  = lastRow['Similar to'] + "," + str(j)
    
    #transitive closure step - Passage en set de la colonne Similar to et deletion des rows sans duplicate
    df['Similar to'] = df['Similar to'].str.split(',')
    for i in range(numberOfRows-1):
        df.at[i,'Similar to'] = set(df.at[i,'Similar to'] )
        df.at[i,'Similar to'].remove('')   
    
    df = df.loc[(df['Similar to'].astype(bool)) ]
    
    

    '''Transitive closure step - pour chaque record on va chercher tous les éléments similaires en allant chercher dans 
    ses éléments similaires puis dans les elements similaires de ses elements simalaires...'''
    for i in df.index:
        closure = set(map(int, df.at[i,'Similar to']))
        while True:
            for dup in closure:
                closure_dup = set(map(int, df.at[dup,'Similar to']))
                new_closure = closure | closure_dup
            
            if new_closure == closure:
                break
            
            closure = new_closure
        
        df.at[i,'Similar to'] = closure
        

    #display(df.iloc[0:800])
    
    #display(df)  
    df.to_excel(outputFile)
        

        
        
        
sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair1,DUPLICATE_FILE)

    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


In [19]:

''' il pourrait être intéressant de dire que l'adress et le name doivent être similaires vu que c'est des attributs souvent uniques au restaurant
On a remarqué que city pouvait être interessant pour confirmer si les restaurants sont identiques '''
def comparePair2(rec1,rec2):
    n1 = set(rec1['name'])
    n2 = set(rec2['name'])
    a1 = set(rec1['address'])
    a2 = set(rec2['address']) 
    c1 = set(rec1['city'])
    c2 = set(rec2['city']) 
    
    dname = nltk.jaccard_distance(n1,n2)
    dadress = nltk.jaccard_distance(a1,a2)
    dcity = nltk.jaccard_distance(c1,c2)
    if dname < 0.2 and dadress < 0.2 and dcity < 0.5:
        return True
    return False


sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair2,"restaurants-duplicates2.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


In [20]:
#règles pour trouver les duplicats avec la distance edit_distance au lieu de jaccard_distance
#on a utilisé edit_distance pour comparer les séquences dans l'objectif de trouver des duplicates
#choix de implémenter comparePair 3: '''En utilisant edit_distance et les champs name et adress
#Les champs city et type ne nous a pas semblé car composé de plusieurs mots
#City pourraient etre pertinent si les deux city sont un peu similaires et que les deux premiers champ sont aussi similaires'''
#si on a un nombre très petit d'opérations , alors on a similarité entre les mots

def comparePair3(rec1,rec2):
    n1 = rec1['name']
    n2 = rec2['name']
    a1 = rec1['address']
    a2 = rec2['address'] 
    dname = nltk.edit_distance(n1,n2)
    dadress = nltk.edit_distance(a1,a2)
    if dname < 5 or dadress < 5:
        return True
    return False

sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair3,"restaurants-duplicates3.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


In [21]:
#module difflib fournit des classes et des fonctions pour comparer les séquences et pour détecter des duplicates 
#Il peut être utilisé, par exemple, pour comparer des fichiers et peut produire des informations 
#sur les différences dans divers formats, y compris HTML et les différences contextuelles et unifiées. 
#on a utilisé difflib.SequenceMatcher et ratio() pour comparer les séquences dans l'objectif de trouver des duplicates
import difflib
#choix de implémenter comparePair 4: '''En utilisant la librairie difflib et les champs name et adress
#Les champs city et type ne nous a pas semblé car composé de plusieurs mots
#City pourraient etre pertinent si les deux city sont un peu similaires et que les deux premiers champ sont aussi similaires'''
#si on a un ratio plus grand que 0.6,alors on a similarité entre les mots
def comparePair4(rec1,rec2):
    n1 = rec1['name']
    n2 = rec2['name']
    a1 = rec1['address']
    a2 = rec2['address'] 
    
    s1 = difflib.SequenceMatcher(lambda x: x == " ",n1,n2)
    s2 = difflib.SequenceMatcher(lambda x: x == " ",a1,a2)
    
    #affichage du ratio 
    #print(round(s1.ratio(),2))
    #print(round(s2.ratio(),2))
    
    if s1.ratio()>0.6 or s2.ratio()>0.6:
        return True
    return False
sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair4,"restaurants-duplicates4.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB


In [22]:

''' On effectue le meme traitement que dans la fonction comparePair2 mais en utilisant les ngrams ce qui semblent plus
restrictif'''
def comparePair5(rec1,rec2):
    n1 = set(nltk.ngrams(rec1['name'], n=3))
    n2 = set(nltk.ngrams(rec2['name'], n=3))
    a1 = set(nltk.ngrams(rec1['address'], n=3))
    a2 = set(nltk.ngrams(rec2['address'], n=3))
    c1 = set(nltk.ngrams(rec1['city'], n=3))
    c2 = set(nltk.ngrams(rec2['city'], n=3))
    
    dname = nltk.jaccard_distance(n1,n2)
    dadress = nltk.jaccard_distance(a1,a2)
    dcity = nltk.jaccard_distance(c1,c2)
    if dname < 0.2 and dadress < 0.2 and dcity < 0.5:
        return True
    return False


sortedNeighboorProgram(SORTED_FILE,WINDOW_SIZE,SHEET_TO_ANALYZE,comparePair5,"restaurants-duplicates5.xlsx")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865 entries, 0 to 864
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        865 non-null    object
 1   address     865 non-null    object
 2   city        865 non-null    object
 3   cuisine     865 non-null    object
 4   Similar to  865 non-null    object
dtypes: object(5)
memory usage: 33.9+ KB
